In [3]:
import io, os, re, sys
import numpy as np
import pandas as pd
import pytesseract
import fitz
import PyPDF2

# Table extraction: Docling(doc:https://docling-project.github.io/docling/), tabula


from dotenv import load_dotenv
from PIL import Image

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            google_api_key=os.getenv("GOOGLE_API_KEY"), 
            temperature=0.1
        )
embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=os.getenv("GOOGLE_API_KEY")
        ) 

In [ ]:
print(sys.version)
print(sys.executable)

In [ ]:
os.listdir('data/')

# Extract data from PDF

In [ ]:
q1_report_path = "data/raw/2025_1Q_LGES_Audit_Report_CONFS_en.pdf"

In [ ]:
docs = fitz.open(q1_report_path)
print(f"total pages: {len(docs)}")

### Docling test

In [ ]:
from docling.document_converter import DocumentConverter

In [ ]:
from pathlib import Path

In [ ]:
# doc_converter = DocumentConverter()
conv_res = doc_converter.convert(q1_report_path)

In [ ]:
doc_filename = conv_res.input.file.stem

In [ ]:
doc_filename

In [ ]:
dfs = {}
for table_i, table in enumerate(conv_res.document.tables):
    table_df: pd.DataFrame = table.export_to_dataframe()
    # print(table_df.to_markdown())
    dfs[f"tabel_{table_i}"] = table_df

----

In [ ]:
print(docs, type(docs))
print(docs[0], type(docs[0]))

In [ ]:
def is_scanned_page(page):
    """
    If pages are covered with only images, run OCR.

    ??? What if half is filled with images and half with text?
    """
    text = page.get_text().strip()
    images = page.get_images()
    return len(images) > 0 and len(text) < 25 

def extract_text_ocr(page):
    pix = page.get_pixmap()
    img_data = pix.tobytes("png")
    img = Image.open(io.BytesIO(img_data))

    # text extraction using OCR
    text = pytesseract.image_to_string(img)
    return text

def clean_text(text):
    """Clean extracted text"""
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters but keep punctuation
    text = re.sub(r'[^\w\s\-.,;:!?()]', '', text)
    return text.strip()

In [ ]:
full_text = ""
for pg_i, page in enumerate(docs):
    if is_scanned_page(page):
        ocr_text = extract_text_ocr(page)
        if ocr_text.strip():
            page_text = ocr_text
        else:
            print(f"--------OCR failed------- at page:{pg_i}")
    else:
        page_text = page.get_text()
        
    page_text = clean_text(page_text)
    full_text += f"\n--- Page {pg_i + 1} ---\n{page_text}\n"
    
with open('data/processed/2025_1Q_LGES_Audit_Report_CONFS_en_fulltext.txt', 'w') as f:
   f.write(full_text) 

In [ ]:
# ??? Extract Table of content: So it's information can be leveraged somewher(ex: chunk by chapter, etc..)
# table_of_contents = doc[1].get_text()

# ??? Better way to extract table? and standardize schema across various reports from diff countries.
# table = doc[5].get_text() + "\n" + doc[6].get_text()

## when PDF contains incomprehensible tables

Unless we use Multi-modal embedding model, or extracting tables work perfectly (Images seem fine but if table in pdf is hard to comprehend, even for human) it is better to add another chain, that is, ask LLM to generate a summary then summary becomes your texts to embed (or could feed it directly).
- Even with Docling, if table inside pdf is not easily comprehensible (even to human), we should either add human in the loop or summarize using multimodal-models.

In [ ]:
text_gen_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.1
)

In [ ]:
# from google import genai
# client = genai.Client()
# myfile = client.files.upload(file=q1_report_path)
# myfile

In [ ]:
from google import genai
from google.genai import types
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

client = genai.Client()
system_message = """
You are a financial analyst who have been following 
LG energy solution for few years.
You are extremely knowlegeable about battery industry.
From given financial report of LG energy solution, 
Create a summary report that will help investors make decisions 
on when to buy on the stock market.
"""

# Upload file
file = client.files.upload(file=q1_report_path)
while file.state.name == 'PROCESSING':
    time.sleep(2)
    file = client.files.get(name=file.name)

# Create cache
# model = 'models/gemini-1.5-flash-latest'
# cache = client.caches.create(
#     model=model,
#     config=types.CreateCachedContentConfig(
#         display_name='Cached Content',
#         system_instruction=(system_message),
#         contents=[file],
#         ttl="300s",
#     )
# )

chain = fin_rep_anal_prmt | text_gen_model

message = HumanMessage(content="")
response = chain.invoke({
    "company_name": 
})

In [ ]:
import base64
from langchain_core.messages import HumanMessage

pdf_bytes = open(q1_report_path, 'rb').read()
pdf_base64 = base64.b64encode(pdf_bytes).decode('utf-8')

message = HumanMessage(
    content=[
        {"type": "text", "text": "You are a financial analyst who have been following LG energy solution for few years. You are extremely knowlegeable about battery industry. From given financial report of LG energy solution, Create a summary report that will help investors make decisions on when to buy on the stock market."},
        {
            "type": "file",
            "source_type": "base64",
            "mime_type":"application/pdf",
            "data": pdf_base64
        }
    ]
)

response = text_gen_model.invoke([message])

In [ ]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


llm_with_tools = text_gen_model.bind_tools([GetWeather, GetPopulation])
ai_msg = llm_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
ai_msg.tool_calls

In [ ]:
ai_msg.content

In [ ]:
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
resp = llm.invoke(
    "When is the next total solar eclipse in US?",
    tools=[GenAITool(google_search={})],
)

# Youtube extractor

In [ ]:
import requests

In [ ]:
CONFIG = {
    'openai_api_key': os.getenv('OPENAI_API_KEY', 'your_openai_key_here'),
    'youtube_api_key': os.getenv('YOUTUBE_API_KEY', ''),  # Optional
    'email_username': os.getenv('EMAIL_USERNAME', 'your_email@gmail.com'),
    'email_password': os.getenv('EMAIL_PASSWORD', 'your_app_password'),
    'check_interval_hours': 6,
    'database_file': 'investment_agent.db'
}

In [ ]:
api_url = "https://www.googleapis.com/youtube/v3/search"

params = {
    'part': 'snippet',
    'channelId': channel_id,
    'maxResults': 5,
    'order': 'date',
    'type': 'video',
    'key': CONFIG['youtube_api_key']
}

response = requests.get(api_url, params=params)
response.raise_for_status()
data = response.json()

In [1]:
from googleapiclient.discovery import build

In [6]:
import google.auth

try:
    credentials, project = google.auth.default()
    print("✅ ADC credentials found and configured.")
    if project:
        print(f"Project ID: {project}")
    else:
        print("No project ID found in credentials.")
except google.auth.exceptions.DefaultCredentialsError as e:
    print(f"❌ ADC credentials not found. Error: {e}")

✅ ADC credentials found and configured.
Project ID: focus-sweep-445012-u8


In [7]:
youtube = build("youtube", "v3")
resp = youtube.search().list(
    q="LG ENERGY solution",
    part='id, snippet',
    maxResults=10,
    type='video'
).execute()

In [9]:
len(resp)

6

In [46]:
video_ids = []
video_titles = []
channel_ids = []
channel_titles = []
pub_dtts = []
n_subs_list = []

for res in resp.get("items")[1:]:
    content_type = res['id']['kind']
    if content_type != 'youtube#video':
        continue
    
    video_id = res['id']['videoId'] # https://www.youtube.com/watch?v={video_id}
    video_title = res['snippet']['title']
    channel_id = res['snippet']['channelId']
    channel_title = res['snippet']['channelTitle']
    published_dtt = res['snippet']['publishTime']
    n_subs = (youtube.channels()
              .list(part='statistics',id=channel_id)
              .execute()['items'][0]['statistics']['subscriberCount'])
    video_ids.append(video_id)
    video_titles.append(video_title)
    channel_ids.append(channel_id)
    channel_titles.append(channel_title)
    pub_dtts.append(published_dtt)
    n_subs_list.append(n_subs)

In [47]:
df = pd.DataFrame({"video_ids":video_ids, "title":video_titles,
                  "channel_title":channel_titles, "pub_dt":pub_dtts, "subs":n_subs_list,
                  "channel_id":channel_ids})

In [53]:
df.dtypes

video_ids                     object
title                         object
channel_title                 object
pub_dt           datetime64[ns, UTC]
subs                          object
channel_id                    object
dtype: object

In [52]:
df['pub_dt']=pd.to_datetime(df['pub_dt'])

In [55]:
df['pub_dt']

0   2025-09-06 01:59:38+00:00
1   2025-09-09 22:10:19+00:00
2   2025-09-07 13:46:00+00:00
3   2025-09-08 05:55:47+00:00
4   2023-02-14 08:00:23+00:00
5   2025-09-05 11:25:21+00:00
6   2024-12-13 08:29:23+00:00
7   2024-01-17 15:53:54+00:00
8   2025-09-06 14:03:24+00:00
Name: pub_dt, dtype: datetime64[ns, UTC]

In [58]:
df['pub_dt'].dt.tz_convert("Asia/Seoul")

0   2025-09-06 10:59:38+09:00
1   2025-09-10 07:10:19+09:00
2   2025-09-07 22:46:00+09:00
3   2025-09-08 14:55:47+09:00
4   2023-02-14 17:00:23+09:00
5   2025-09-05 20:25:21+09:00
6   2024-12-13 17:29:23+09:00
7   2024-01-18 00:53:54+09:00
8   2025-09-06 23:03:24+09:00
Name: pub_dt, dtype: datetime64[ns, Asia/Seoul]

In [48]:
df.loc[df['subs'] > 100_000]

,video_ids,title,channel_title,pub_dt,subs,channel_id
0,ggOFAzg_1kU,Georgia ICE raid: Hundreds detained at Hyundai...,LiveNOW from FOX,2025-09-06T01:59:38Z,5130000,UCJg9wBPyKMNA5sRDnvzmkdg
1,wHP_95GXkm0,Medical calls continue at LG Energy plant days...,WOOD TV8,2025-09-09T22:10:19Z,256000,UCDG0mJ_zkrODDT-4VTEV6TQ
2,jOZTyrxw4_M,Official: South Korean workers detained in Geo...,CNN,2025-09-07T13:46:00Z,18700000,UCupvZG-5ko_eiXAupbDfxWw
3,hftATC9_Dwk,US Raid on Hyundai Plant Leaves Korean Compani...,Bloomberg Television,2025-09-08T05:55:47Z,2840000,UCIALMKvObZNtJ6AmdCLP7Lg
4,_agkocyC9e8,LG에너지솔루션 브로츠와프 법인 직원들의 DAY 2 ㅣ V-log Tour DAY ...,LG에너지솔루션 [LG Energy Solution],2023-02-14T08:00:23Z,246000,UCEtI8QbOcwcCsOFt-kgr2gg
5,wIPACSQV_eo,"US Raids Hyundai-LG Battery Plant, 300 Koreans...",MBN News,2025-09-05T11:25:21Z,2490000,UCG9aFJTZ-lMCHAiO1KJsirg
6,uwfK9uHJQW4,EMPOWER EVERY POSSIBILITY | LG Energy Solution,LG Energy Solution Global,2024-12-13T08:29:23Z,95600,UCmkjW0ZnWqw0n6PSXYr_v5A
7,d8ne6uE-50Y,About LG Energy Solution Michigan,"LG Energy Solution Michigan, Inc.",2024-01-17T15:53:54Z,257,UC5bG-hCQQ4MLjSGs5CDeHtQ
8,S4t1aF79dVQ,한국 기업 단속 때 헬기까지 출동…손발 체인 묶어 줄줄이 연행,중앙일보,2025-09-06T14:03:24Z,555000,UCH3mJ-nHxjjny2FJbJaqiDA


In [29]:
res.keys()

dict_keys(['kind', 'etag', 'id', 'snippet'])

In [37]:
resp.get("items")[0]

{'kind': 'youtube#searchResult',
 'etag': 'zROkztDtIrjJnuAgwC4fTWbV71o',
 'id': {'kind': 'youtube#channel', 'channelId': 'UCEtI8QbOcwcCsOFt-kgr2gg'},
 'snippet': {'publishedAt': '2020-12-03T02:09:05Z',
  'channelId': 'UCEtI8QbOcwcCsOFt-kgr2gg',
  'title': 'LG에너지솔루션 [LG Energy Solution]',
  'description': 'EMPOWER EVERY POSSIBILITY 에너지에 대한 다양한 솔루션을 제공하는 LG에너지솔루션 공식 채널입니다.',
  'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/GaZex5gyFhFug_Q7Kan8JTUqRZxQ-oXVGyhzQYCtB1E_6d-7JMRZUG-q9_r50MKYVW5g0-vYZg=s88-c-k-c0xffffffff-no-rj-mo'},
   'medium': {'url': 'https://yt3.ggpht.com/GaZex5gyFhFug_Q7Kan8JTUqRZxQ-oXVGyhzQYCtB1E_6d-7JMRZUG-q9_r50MKYVW5g0-vYZg=s240-c-k-c0xffffffff-no-rj-mo'},
   'high': {'url': 'https://yt3.ggpht.com/GaZex5gyFhFug_Q7Kan8JTUqRZxQ-oXVGyhzQYCtB1E_6d-7JMRZUG-q9_r50MKYVW5g0-vYZg=s800-c-k-c0xffffffff-no-rj-mo'}},
  'channelTitle': 'LG에너지솔루션 [LG Energy Solution]',
  'liveBroadcastContent': 'none',
  'publishTime': '2020-12-03T02:09:05Z'}}

In [ ]:
['id']['kind']

In [34]:
resp.get("items")[1]

{'kind': 'youtube#searchResult',
 'etag': 'NjyTqsYWLa_g3i4nbI71Ui5No8E',
 'id': {'kind': 'youtube#video', 'videoId': 'ggOFAzg_1kU'},
 'snippet': {'publishedAt': '2025-09-06T01:59:38Z',
  'channelId': 'UCJg9wBPyKMNA5sRDnvzmkdg',
  'title': 'Georgia ICE raid: Hundreds detained at Hyundai factory | LiveNOW from FOX',
  'description': 'Federal authorities say 475 people were detained this week in what Homeland Security Investigations called the largest ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/ggOFAzg_1kU/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/ggOFAzg_1kU/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/ggOFAzg_1kU/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'LiveNOW from FOX',
  'liveBroadcastContent': 'none',
  'publishTime': '2025-09-06T01:59:38Z'}}

## Sending video or youtube to Gemini

In [ ]:
video_bytes = open("/path/to/your/video.mp4", 'rb').read()
video_base64 = base64.b64encode(video_bytes).decode('utf-8')

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe what's in this video in a sentence"},
        {
            "type": "file",
            "source_type": "base64",
            "mime_type": "video/mp4",
            "data": video_base64
        }
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content

message = HumanMessage(
    content=[
        {"type": "text", "text": "summarize the video in 3 sentences."},
        {
            "type": "media",
            "file_uri": "https://www.youtube.com/watch?v=9hE5-98ZeCg",
            "mime_type": "video/mp4",
        }
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content